In [180]:
import pygmsh
import numpy as np
import meshio

#### read in gmesh file (before partition)

In [181]:
msh = meshio.read("ell1.msh")

In [182]:
for cell in msh.cells:
    if cell.type == "triangle":
        triangle_cells = cell.data
    elif  cell.type == "tetra":
        tetra_cells = cell.data
    elif  cell.type == "hexahedron":
        hexahedron_cells = cell.data
    elif  cell.type == "quad":
        quad_cells = cell.data

#### write mesh to file with METIS consumable mesh format (before partition)

In [183]:
# METIS mesh format
#  ==== METIS 4 ====
#  METIS type code  meshio cell type    details                        notes
#  1                triangle            2D triangular elements         (vertices can be listed in any order)
#  2                tetra               3D tetrahedral elements        (vertices can be listed in any order)
#  3                hexahedron          3D hexahedral (brick) elements (vertices must be listed in a particular order)
#  4                quad                2D quadrilateral elements.     (vertices must be listed in a particular order)

#  ==== METIS 5 ====
# METIS type code should be 1, see manual
metis_version = 5

In [184]:
type_map = {'triangle':1, 'tetra':2, 'hexahedron':3, 'quad':4}
type_list = [type_map.get(x) for x in list(msh.cells_dict.keys()) if type_map.get(x) is not None]
if len(type_list) == 1:
    mesh_type = type_list[0]
    cells = msh.cells_dict[ list(type_map.keys())[list(type_map.values()).index(mesh_type)] ]
    if metis_version > 4:
        mesh_type = 1
else:
    raise Exception("mixed element mesh?")

In [185]:
## convert from 0 indexing to 1 indexing
cells += 1

In [186]:
with open("ell.mesh", "wb") as f:
    #  The first line lists the number of elements, and their type.
    f.write(' '.join([str(i) for i in [len(cells), mesh_type]]).encode('ascii'))
    f.write(b"\n")
    
    # The rest lines contain the element node array
    np.savetxt(f, cells, fmt = '%d', delimiter = ' ')

#### read in partition result

In [162]:
#part_file = "/Users/xgai/Downloads/ell/dual/ell.mesh.epart.4"
#part_file ="/Users/xgai/Downloads/cube/cube.mesh.epart.8"

In [163]:
with open(part_file, "r") as f:
    part = [int(i) for i in f.read().split('\n') if i != '']

In [165]:
def save_triangle_part_obj(name, part, msh):
    # add path
    for part_id in set(part):
        triangle_mesh =meshio.Mesh(points=msh.points, cells=[("triangle", triangle_cells[[i == part_id for i in part]])])
        meshio.write(name + str(part_id) + ".obj", triangle_mesh, "obj")

In [169]:
def save_tetra_part_obj(name, part, msh):
    # add path
    for part_id in set(part):
        tetra_mesh =meshio.Mesh(points=msh.points, cells=[("tetra", tetra_cells[[i == part_id for i in part]])])
        #meshio.write(name + str(part_id) + ".obj", tetra_mesh, "obj")
        meshio.write(name + str(part_id) + ".vtk", tetra_mesh, "vtk")

In [170]:
#save_triangle_part_obj('test', part, msh)
save_tetra_part_obj('test', part, msh) # for 3d, load all the parts to paraview and assign different color

In [33]:
# # lines and points can not be written to .obj file
# triangle_mesh =meshio.Mesh(points=msh.points, cells=[("triangle", triangle_cells)])
# meshio.write("tri0.obj", triangle_mesh0,"obj")

#### read the obj file generated by meshio for visualization

In [173]:
import open3d as o3d

In [174]:
# # read in coordinates file and draw vertices
# pcd = o3d.io.read_point_cloud("/Users/xgai/Downloads/ell/ell.xyz", format='xyz')
# np.asarray(pcd.points)
# o3d.visualization.draw_geometries([pcd])

In [175]:
def read_objs_for_visual(name, part):
    mesh_lists = []
    for part_id in set(part):
        tmp_mesh = o3d.io.read_triangle_mesh(name + str(part_id)+ ".obj")
        tmp_mesh.paint_uniform_color([np.random.uniform(), np.random.uniform(), np.random.uniform()])
        mesh_lists.append(tmp_mesh)
    return mesh_lists

In [177]:
meshes = read_objs_for_visual('test', part)
o3d.visualization.draw_geometries(meshes, mesh_show_wireframe =True)

[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.
[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.
[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.
[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.
[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.
[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.
[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.
[Open3D WARNING] Read geometry::TriangleMesh failed: unknown file extension.


RuntimeError: [1;31m[Open3D ERROR] [CreateCoordinateFrame] size <= 0[0;m

In [88]:
# # read in objfile
# mesh = o3d.io.read_triangle_mesh("test.obj")
# #print(mesh)
# # get vertices coords and element node array
# np.asarray(mesh.vertices)
# np.asarray(mesh.triangles)
# # color the mesh
# tmp_mesh.paint_uniform_color([1,0,0])
# o3d.visualization.draw_geometries([mesh], mesh_show_wireframe =True)

array([[0.        , 0.        , 0.        ],
       [2.        , 0.        , 0.        ],
       [2.        , 1.        , 0.        ],
       [1.        , 1.        , 0.        ],
       [1.        , 2.        , 0.        ],
       [0.        , 2.        , 0.        ],
       [0.5       , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.5       , 0.        , 0.        ],
       [2.        , 0.5       , 0.        ],
       [1.5       , 1.        , 0.        ],
       [1.        , 1.5       , 0.        ],
       [0.5       , 2.        , 0.        ],
       [0.        , 1.5       , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.5       , 0.        ],
       [0.6248979 , 0.6248979 , 0.        ],
       [0.36519608, 0.89460784, 0.        ],
       [0.89460784, 0.36519608, 0.        ],
       [0.49609011, 1.29651022, 0.        ],
       [1.29651022, 0.47823295, 0.        ],
       [0.37260574, 0.37260574, 0.        ],
       [0.